In [ ]:
import subprocess
import sys
import site

def install_package_and_fix_import(package_name):
    try:
        # Create a virtual environment
        subprocess.run(["python3", "-m", "venv", "myenv"])

        # Activate the virtual environment
        activate_script = "myenv/bin/activate"
        subprocess.run([activate_script], shell=True)

        # Install the package within the virtual environment
        subprocess.run(["pip", "install", package_name])

        # Get the site-packages directory of the virtual environment
        venv_site_packages = site.getsitepackages()[0]

        # Add the site-packages directory to sys.path
        sys.path.append(venv_site_packages)

        print(f"Package {package_name} installed and sys.path updated successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")


package_to_install = "bitsandbytes"
install_package_and_fix_import(package_to_install)
package_to_install = "datasets"
install_package_and_fix_import(package_to_install)
package_to_install = "accelerate"
install_package_and_fix_import(package_to_install)
package_to_install = "loralib"
install_package_and_fix_import(package_to_install)
package_to_install = "transformers"
install_package_and_fix_import(package_to_install)
package_to_install = "git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git"
install_package_and_fix_import(package_to_install)
package_to_install = "torch"
install_package_and_fix_import(package_to_install)

In [ ]:
package_to_install = "os"
install_package_and_fix_import(package_to_install)

In [ ]:
package_to_install = "peft"
install_package_and_fix_import(package_to_install)

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloomz-3b", #tokenizer for all bloom models
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-3b")

In [ ]:
print(model) #30 bloom block which we will target

In [20]:
#freezing weights and to ensure stability
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [23]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],#the attention layer block to which we want to target
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)#the lorafied model
#because of lora we have to train only 4925440 out of all params: 3007482880 isnt it crazy less than percent

In [24]:
import transformers
from datasets import load_dataset

In [ ]:
dataset = load_dataset('FourthBrainGenAI/MarketMail-AI')
print(dataset)
print(dataset['train'][0])

In [ ]:
def generate_prompt(product: str, description: str, marketing_email: str) -> str:
  prompt = f"### INSTRUCTION\nBelow is a product and description, please write a marketing email for this product.\n\n### Product:\n{product}\n### Description:\n{description}\n\n### Marketing Email:\n{marketing_email}"
  return prompt

mapped_dataset = dataset.map(lambda samples: tokenizer(generate_prompt(samples['product'], samples['description'], samples['marketing_email'])))


In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=100,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()


In [ ]:
from huggingface_hub import notebook_login
notebook_login()#make sure to pass the hubbing face token in write mode

In [ ]:
model_name = "rohit-bloom-finetuned"
HUGGING_FACE_USER_NAME = "rjindal"
model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)

In [ ]:
model_name = "rohit-bloom-finetuned"
HUGGING_FACE_USER_NAME = "rjindal"
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
from IPython.display import display, Markdown

def make_inference(product, description):
  batch = tokenizer(f"### INSTRUCTION\nBelow is a product and description, please write a marketing email for this product.\n\n### Product:\n{product}\n### Description:\n{description}\n\n### Marketing Email:\n", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=200)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))


In [ ]:
your_product_name_here = "Iwatch"
your_product_description_here = "mobile phone on your wrist!"

make_inference(your_product_name_here, your_product_description_here)